## **Data Cleaning Summary**

The dataset was cleaned to improve consistency, usability, and reliability for analysis. The following steps were performed:

1. **Column Name Standardization**  
   - All column names were converted to lowercase.
   - Spaces were replaced with underscores to follow standard naming conventions.
   - This ensures easier handling of columns in Python, SQL, and BI tools.


2. **Standardization of `item_fat_content` Values**  
   - The `item_fat_content` column contained inconsistent labels such as  
     `Regular`, `reg`, `Low Fat`, `low fat`, and `LF`.
   - These values were standardized into two consistent categories:  
     **Regular** and **Low Fat**.


3. **Handling Missing Values in `item_weight`**  
   - The `item_weight` column contained missing values (~17% of rows).
   - A two-level imputation strategy was applied:
     - First, missing values were filled using the **median item weight for the same `item_identifier`**.
     - Remaining missing values were filled using the **median item weight of the corresponding `item_type`**.
   - This approach preserves item-level accuracy while providing a reasonable fallback.
   - After imputation, no missing values remained in the `item_weight` column.


4. **Final Data Quality Check**  
   - The dataset contains no missing values in critical analytical columns.
   - Categorical variables are consistent and standardized.
   - The dataset is now suitable for exploratory data analysis, SQL-based analysis, and dashboard development.

---

### Cleaning Outcome
After cleaning, the dataset is structured, consistent, and analysis-ready, closely reflecting real-world business data used in retail analytics workflows.

## **Import Libraries**

In [3]:
import pandas as pd
import numpy as np 

## **Dataset Load**

In [5]:
data = pd.read_excel(r"C:\Users\rudra\OneDrive\Documents\GitHub\blinkit-end-to-end-data-analysis\data\raw\blinkit_grocery_data.xlsx")
df = data.copy()

In [6]:
df.head()

,Item Fat Content,Item Identifier,Item Type,Outlet Establishment Year,Outlet Identifier,Outlet Location Type,Outlet Size,Outlet Type,Item Visibility,Item Weight,Sales,Rating
0,Regular,FDX32,Fruits and Vegetables,2012,OUT049,Tier 1,Medium,Supermarket Type1,0.100014,15.10,145.4786,5.0
1,Low Fat,NCB42,Health and Hygiene,2022,OUT018,Tier 3,Medium,Supermarket Type2,0.008596,11.80,115.3492,5.0
2,Regular,FDR28,Frozen Foods,2016,OUT046,Tier 1,Small,Supermarket Type1,0.025896,13.85,165.0210,5.0
3,Regular,FDL50,Canned,2014,OUT013,Tier 3,High,Supermarket Type1,0.042278,12.15,126.5046,5.0
4,Low Fat,DRI25,Soft Drinks,2015,OUT045,Tier 2,Small,Supermarket Type1,0.033970,19.60,55.1614,5.0


## **Data Cleaning**

### **1. Standardizing Column Name**

In [9]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

In [10]:
df.sample(5)

,item_fat_content,item_identifier,item_type,outlet_establishment_year,outlet_identifier,outlet_location_type,outlet_size,outlet_type,item_visibility,item_weight,sales,rating
7325,Regular,DRB13,Soft Drinks,2020,OUT017,Tier 2,Medium,Supermarket Type1,0.007084,6.115,191.1530,4.0
280,Low Fat,DRD60,Soft Drinks,2017,OUT035,Tier 2,Small,Supermarket Type1,0.037225,15.700,182.4634,5.0
6880,Low Fat,FDC04,Dairy,2015,OUT045,Tier 2,Medium,Supermarket Type1,0.045077,15.600,241.3854,4.0
7158,Low Fat,FDD35,Starchy Foods,2020,OUT017,Tier 2,High,Supermarket Type1,0.026011,12.150,119.1440,4.0
5180,Low Fat,NCK29,Health and Hygiene,2012,OUT049,Tier 1,Medium,Supermarket Type1,0.125965,5.615,124.7730,3.1


### **2. Fixing the Inonsistency in `item_fat_content` column**

In [12]:
df['item_fat_content'].unique()

array(['Regular', 'Low Fat', 'low fat', 'LF', 'reg'], dtype=object)

In [13]:
replace_map = {

    "reg" : "Regular",
    "low fat" : "Low Fat",
    "LF" : "Low Fat"
}

df['item_fat_content'] = df['item_fat_content'].replace(replace_map)

In [14]:
df['item_fat_content'].unique()

array(['Regular', 'Low Fat'], dtype=object)

In [15]:
df.sample(5)

,item_fat_content,item_identifier,item_type,outlet_establishment_year,outlet_identifier,outlet_location_type,outlet_size,outlet_type,item_visibility,item_weight,sales,rating
814,Low Fat,FDN20,Fruits and Vegetables,2020,OUT017,Tier 2,Small,Supermarket Type1,0.026330,19.350,169.2474,4.5
3217,Regular,FDQ48,Baking Goods,2017,OUT035,Tier 2,Small,Supermarket Type1,0.034405,14.300,98.1726,4.0
5326,Low Fat,FDS23,Breads,2017,OUT035,Tier 2,Small,Supermarket Type1,0.140862,4.635,126.4994,3.0
4014,Low Fat,FDK27,Meat,2017,OUT035,Tier 2,Small,Supermarket Type1,0.008945,11.000,122.3756,3.9
3709,Low Fat,FDS01,Canned,2022,OUT018,Tier 3,Medium,Supermarket Type2,0.017817,11.600,177.0686,4.0


### **3. Fixing Missing values in the `item_weight` column**

In [17]:
df['item_weight'].isnull().sum()

1463

In [18]:
df['item_weight'] = df.groupby('item_identifier')['item_weight'].transform(lambda x: x.fillna(x.median()))

In [19]:
df['item_weight'].isnull().sum()

4

In [20]:
df[df['item_weight'].isnull()]

,item_fat_content,item_identifier,item_type,outlet_establishment_year,outlet_identifier,outlet_location_type,outlet_size,outlet_type,item_visibility,item_weight,sales,rating
2046,Regular,FDN52,Frozen Foods,2018,OUT027,Tier 3,Medium,Supermarket Type3,0.130933,NaN,86.9198,4.2
4582,Low Fat,FDK57,Snack Foods,2018,OUT027,Tier 3,Medium,Supermarket Type3,0.079904,NaN,120.0440,3.7
6514,Regular,FDQ60,Baking Goods,2018,OUT019,Tier 1,Small,Grocery Store,0.191501,NaN,121.2098,4.0
8452,Regular,FDE52,Dairy,2018,OUT027,Tier 3,Medium,Supermarket Type3,0.029742,NaN,88.9514,4.0


In [21]:
df ['item_weight'] = df.groupby('item_type')['item_weight'].transform(lambda x: x.fillna(x.median()))

In [22]:
df['item_weight'].isnull().sum()

0

In [23]:
df.sample(5)

,item_fat_content,item_identifier,item_type,outlet_establishment_year,outlet_identifier,outlet_location_type,outlet_size,outlet_type,item_visibility,item_weight,sales,rating
7203,Regular,FDO52,Frozen Foods,2015,OUT045,Tier 2,High,Supermarket Type1,0.077321,11.600,169.3106,4.0
6807,Regular,FDX33,Snack Foods,2016,OUT046,Tier 1,Small,Supermarket Type1,0.117485,9.195,158.5578,4.0
821,Regular,FDU50,Dairy,2015,OUT045,Tier 2,High,Supermarket Type1,0.075323,5.750,116.3176,4.5
848,Low Fat,DRL60,Soft Drinks,2017,OUT035,Tier 2,Small,Supermarket Type1,0.027054,8.520,151.5682,4.5
559,Low Fat,FDK51,Dairy,2011,OUT010,Tier 3,Medium,Grocery Store,0.008763,19.850,266.6884,4.7


In [24]:
df.isnull().sum()

item_fat_content             0
item_identifier              0
item_type                    0
outlet_establishment_year    0
outlet_identifier            0
outlet_location_type         0
outlet_size                  0
outlet_type                  0
item_visibility              0
item_weight                  0
sales                        0
rating                       0
dtype: int64

In [25]:
## **Exporting Cleanned Data**

In [26]:
df.to_excel(r"C:\Users\rudra\OneDrive\Documents\GitHub\blinkit-end-to-end-data-analysis\data\processed\cleanned_blinkit_grocery_data.xlsx", index = False)
print("Data Exported Successfully.")

Data Exported Successfully.
